# Import

In [3]:
import pandas as pd

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

# from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# import torch

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 1번 GPU만 노출됨

# Data Load & Pre-processing

In [5]:
train = pd.read_csv('../../data/train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('../../data/test.csv', encoding = 'utf-8-sig')

FileNotFoundError: [Errno 2] No such file or directory: '../../data/train.csv'

In [10]:
train.columns

Index(['ID', '발생일시', '사고인지 시간', '날씨', '기온', '습도', '공사종류', '연면적', '층 정보',
       '인적사고', '물적사고', '공종', '사고객체', '작업프로세스', '장소', '부위', '사고원인',
       '재발방지대책 및 향후조치계획', '공사종류(대분류)', '공사종류(중분류)', '공종(대분류)', '공종(중분류)',
       '사고객체(대분류)', '사고객체(중분류)'],
      dtype='object')

In [9]:
train.head()

,ID,발생일시,사고인지 시간,날씨,기온,습도,공사종류,연면적,층 정보,인적사고,...,장소,부위,사고원인,재발방지대책 및 향후조치계획,공사종류(대분류),공사종류(중분류),공종(대분류),공종(중분류),사고객체(대분류),사고객체(중분류)
0,TRAIN_00000,2023-12-31 오후 12:44,정규작업 -,맑음,1℃,30%,건축 / 건축물 / 근린생활시설,"4,892.77㎡","지상 14층, 지하 3층",떨어짐(5미터 이상 ~ 10미터 미만),...,근린생활시설 / 내부,철근 / 고소,"고소작업 중 추락 위험이 있음에도 불구하고, 안전난간대, 안전고리 착용 등 안전장치...",고소작업 시 추락 위험이 있는 부위에 안전장비 설치.,건축,건축물,건축,철근콘크리트공사,건설자재,철근
1,TRAIN_00001,2023-12-30 오후 03:35,정규작업 -,맑음,10℃,90%,토목 / 터널 / 철도터널,-,-,끼임,...,철도터널 / 내부,볼트 / 바닥,부주의,재발 방지 대책 마련과 안전교육 실시.,토목,터널,토목,터널공사,건설자재,볼트
2,TRAIN_00002,2023-12-30 오후 02:30,정규작업 -,맑음,14℃,70%,건축 / 건축물 / 업무시설,"1,994.62㎡","지상 5층, 지하 0층",넘어짐(미끄러짐),...,업무시설 / 내부,기타 / 바닥,3층 슬라브 작업시 이동중 미끄러짐,현장자재 정리와 안전관리 철저를 통한 재발 방지 대책 및 공문 발송을 통한 향후 조...,건축,건축물,건축,철근콘크리트공사,기타,기타
3,TRAIN_00003,2023-12-30 오후 12:00,휴일근무 -,흐림,12℃,55%,토목 / 하천 / 기타,-,-,기타,...,기타 / 동산교 신축구간,교각 기초 / 바닥,"교각 기초철근 조립 중 강한 바람에 의해 기둥측 주철근이 균형을 잃고 전도되어, 하...","위험성 평가 및 교육을 통해 작업장 내 위험요인과 안전수칙을 근로자에게 전파하고, ...",토목,하천,토목,하천공사,부재,교각 기초
4,TRAIN_00004,2023-12-30 오전 10:00,정규작업 -,맑음,0℃,10%,건축 / 건축물 / 공동주택,"59,388.93㎡","지상 27층, 지하 3층",넘어짐(미끄러짐),...,공동주택 / 내부,건설폐기물 / 바닥,근로자의 부주의,자재 정리 작업 시 세부 작업 방법에 대한 교육 실시와 작업 구간 이동 경로 점검 ...,건축,건축물,건축,해체 및 철거공사,기타,건설폐기물


In [4]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]

In [5]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [6]:
# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

# Model import

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16   
)

In [7]:
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Vector store 생성

In [8]:
combined_training_data.head()

,question,answer
0,"공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류...",고소작업 시 추락 위험이 있는 부위에 안전장비 설치.
1,"공사종류 대분류 '토목', 중분류 '터널' 공사 중 공종 대분류 '토목', 중분류 ...",재발 방지 대책 마련과 안전교육 실시.
2,"공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류...",현장자재 정리와 안전관리 철저를 통한 재발 방지 대책 및 공문 발송을 통한 향후 조...
3,"공사종류 대분류 '토목', 중분류 '하천' 공사 중 공종 대분류 '토목', 중분류 ...","위험성 평가 및 교육을 통해 작업장 내 위험요인과 안전수칙을 근로자에게 전파하고, ..."
4,"공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류...",자재 정리 작업 시 세부 작업 방법에 대한 교육 실시와 작업 구간 이동 경로 점검 ...


In [8]:
# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}" 
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]

# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-nli"  # 임베딩 모델 선택
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
csv_vector_store = FAISS.from_texts(train_documents, embedding)

# 1. 임베딩 모델과 Chroma 벡터 스토어 로드 (PDF 문서 기반)
embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-nli")
pdf_vector_store = Chroma(
    persist_directory="../../data/chroma_construction_db_v2", 
    embedding_function=embedding
)

/tmp/ipykernel_3230035/3023847848.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)
/tmp/ipykernel_3230035/3023847848.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  pdf_vector_store = Chroma(


In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.document_loaders import PyPDFLoader
# import os
# import pandas as pd

# ###  PDF 데이터 처리 및 Vector Store 저장
# def load_pdfs_from_folder(pdf_folder):
#     """폴더 내 모든 PDF 문서를 LangChain을 통해 로드"""
#     pdf_documents = []
    
#     for pdf_file in os.listdir(pdf_folder):
#         if pdf_file.endswith(".pdf"):
#             pdf_path = os.path.join(pdf_folder, pdf_file)
#             loader = PyPDFLoader(pdf_path)  # LangChain PDF 로더
#             pdf_documents.extend(loader.load())  # 문서 추가

#     return pdf_documents

# # PDF 데이터 로드
# pdf_folder_path = "../../data/pdf"
# pdf_docs = load_pdfs_from_folder(pdf_folder_path)

# # 문서 분할 (Chunking) 수행
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500, chunk_overlap=100, separators = ['\n\n', '\n', '.', ' ']
# )
# split_pdf_docs = text_splitter.split_documents(pdf_docs)

# # PDF 데이터를 벡터화하여 FAISS 저장소 생성
# pdf_vector_store = FAISS.from_documents(split_pdf_docs, embedding)

# kanana ?

In [ ]:
# # 임베딩 생성
# embedding_model_name = "kakaocorp/kanana-nano-2.1b-embedding"  # 임베딩 모델 선택
# embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# RAG chain 생성

In [ ]:
# import numpy as np
# import string
# from langchain.llms import Ollama
# from langchain.prompts import PromptTemplate
# from langchain.chains import RetrievalQA

# # Cosine Similarity 산식
# def cosine_similarity(a, b):
#     dot_product = np.dot(a, b)
#     norm_a = np.linalg.norm(a)
#     norm_b = np.linalg.norm(b)
#     return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

# # 텍스트 정규화 함수: 소문자화 및 구두점 제거
# def normalize_text(text):
#     text = text.lower()
#     text = text.translate(str.maketrans('', '', string.punctuation))
#     return text.strip()

# # 후보 답변들 간의 평균 자카드 유사도를 계산하는 함수
# def average_similarity(candidate, candidates):
#     total = 0
#     count = 0
#     for other in candidates:
#         if candidate == other:
#             continue
#         total += cosine_similarity(candidate, other)
#         count += 1
#     return total / count if count > 0 else 0

# # LLM 설정 (Ollama 사용)
# llm = Ollama(model="gemma3:27b", temperature=0)

# # 개선된 프롬프트 템플릿
# prompt_template = """
# ### 지침: 당신은 건설 안전 전문가입니다.
# 답변은 반드시 핵심 용어와 주요 문구만을 사용하여 작성해야 합니다.
# - 서론, 배경 설명, 부연 설명은 절대 포함하지 마세요.
# - 최대 64 토큰 이내로 간결하게 작성하세요.
# - 평가 기준(코사인 유사도)을 높이기 위해 정답과 동일하거나 유사한 단어를 사용하세요.
# 예시: "안전관리 시스템 강화, 사고 예방 프로토콜 개선"

# {context}

# ### 질문:
# {question}

# [/INST]
# """

# # 커스텀 프롬프트 생성
# prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template=prompt_template,
# )

# # Retriever 설정
# retriever = csv_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# # RAG 체인 생성
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,  
#     chain_type="stuff",  
#     retriever=retriever,  
#     return_source_documents=True,
#     chain_type_kwargs={"prompt": prompt}
# )

In [9]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import EnsembleRetriever
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# 2. 각 소스별 retriever 생성 (각 소스에서 상위 3개 문서 검색)
pdf_retriever = pdf_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
train_retriever = csv_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 3. 두 retriever를 EnsembleRetriever로 결합 (PDF에 70%, Train Q&A에 30% 가중치 부여)
ensemble_retriever = EnsembleRetriever(
    retrievers=[pdf_retriever, train_retriever],
    weights=[0.7, 0.3]
)

# 4. RetrievalQA 체인에 사용할 프롬프트 템플릿 정의
# refine 체인이 아닌 "stuff" 체인을 사용하며, 문서 전달 변수명을 "context_str"로 맞춥니다.
prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
테스트 데이터에 주어진 사고 상황에 대해, 검색된 문맥을 참고하여 핵심 대책을 간결하게 작성하세요.
- 서론, 부연 설명 없이 핵심 단어와 문구만 포함합니다.
- 최대 64 토큰 이내로 간결하게 작성하세요.
- 예시: "안전관리 시스템 강화, 사고 예방 프로토콜 개선"

{context_str}

### 질문:
{question}

[/INST]
"""
prompt = PromptTemplate(
    input_variables=["context_str", "question"],
    template=prompt_template,
)

# 5. LLM 및 RetrievalQA 체인 생성
llm = Ollama(model="gemma3:27b", temperature=0)
qa_chain_hybrid = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  
    retriever=ensemble_retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt,
        "document_variable_name": "context_str"  # 프롬프트에서 사용한 변수명과 일치시킵니다.
    }
)

/tmp/ipykernel_3230035/1546010801.py:40: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3:27b", temperature=0)


# Inference

In [ ]:
# # 각 질문에 대해 여러 후보 답변을 생성하고, 후보들 간 평균 유사도가 가장 높은 답변을 최종 선택
# num_candidates = 3  # 각 질문당 후보 답변 개수
# test_results = []

# print("테스트 실행 시작... 총 테스트 샘플 수:", len(combined_test_data))

# for idx, row in combined_test_data.iterrows():
#     if (idx + 1) % 10 == 0 or idx == 0:
#         print(f"\n[샘플 {idx + 1}/{len(combined_test_data)}] 진행 중...")
    
#     question = row['question']
    
#     # 여러 후보 답변 생성
#     candidate_answers = []
#     for _ in range(num_candidates):
#         result = qa_chain.invoke(question)
#         candidate = normalize_text(result['result'])
#         candidate_answers.append(candidate)
    
#     # 후보들 간의 평균 유사도를 계산하여 가장 일관성 높은 후보 선택
#     avg_similarities = [average_similarity(candidate, candidate_answers) for candidate in candidate_answers]
#     best_candidate = candidate_answers[np.argmax(avg_similarities)]
    
#     test_results.append(best_candidate)

# print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

In [ ]:
test_results = []

print("테스트 실행 시작... 총 테스트 샘플 수:", len(combined_test_data))
for idx, row in combined_test_data.iterrows():
    if (idx + 1) % 10 == 0 or idx == 0:
        print(f"\n[샘플 {idx + 1}/{len(combined_test_data)}] 진행 중...")
    
    question = row['question']  # test.csv의 'question' 컬럼 사용
    result = qa_chain_hybrid.invoke(question)
    final_answer = result['result'].strip()
    test_results.append(final_answer)

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 964

[샘플 1/964] 진행 중...

[샘플 10/964] 진행 중...

[샘플 20/964] 진행 중...

[샘플 30/964] 진행 중...

[샘플 40/964] 진행 중...

[샘플 50/964] 진행 중...

[샘플 60/964] 진행 중...

[샘플 70/964] 진행 중...

[샘플 80/964] 진행 중...

[샘플 90/964] 진행 중...

[샘플 100/964] 진행 중...

[샘플 110/964] 진행 중...

[샘플 120/964] 진행 중...

[샘플 130/964] 진행 중...

[샘플 140/964] 진행 중...

[샘플 150/964] 진행 중...

[샘플 160/964] 진행 중...

[샘플 170/964] 진행 중...

[샘플 180/964] 진행 중...

[샘플 190/964] 진행 중...

[샘플 200/964] 진행 중...

[샘플 210/964] 진행 중...

[샘플 220/964] 진행 중...

[샘플 230/964] 진행 중...

[샘플 240/964] 진행 중...

[샘플 250/964] 진행 중...

[샘플 260/964] 진행 중...

[샘플 270/964] 진행 중...

[샘플 280/964] 진행 중...

[샘플 290/964] 진행 중...

[샘플 300/964] 진행 중...

[샘플 310/964] 진행 중...

[샘플 320/964] 진행 중...

[샘플 330/964] 진행 중...

[샘플 340/964] 진행 중...

[샘플 350/964] 진행 중...

[샘플 360/964] 진행 중...

[샘플 370/964] 진행 중...

[샘플 380/964] 진행 중...

[샘플 390/964] 진행 중...

[샘플 400/964] 진행 중...

[샘플 410/964] 진행 중...

[샘플 420/964] 진행 중...

[샘플 430/964] 진행 중...

[샘플 440/964] 

In [31]:
# 중복 여부 확인: 리스트의 길이와 집합의 길이를 비교
if len(test_results) != len(set(test_results)):
    print("중복값이 있습니다.")
else:
    print("중복값이 없습니다.")

# 중복된 값들을 구체적으로 출력하기 (collections.Counter 사용)
from collections import Counter
duplicates = [f'{item}, {count}' for item, count in Counter(test_results).items() if count > 1]
print("중복된 항목:", duplicates)


중복값이 있습니다.
중복된 항목: ['작업자 안전교육 실시 보안면 착용 확인 숫돌 사용법 교육 및 감독 현장 안전점검 강화, 2', '안전교육 강화 및 작업 전 안전 점검 철저, 3', ', 4', '작업자 건강 상태 점검 및 근골격계 질환 예방 교육 강화, 2', '재발 방지 대책 및 향후 조치 계획, 2']


# Submission

In [47]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

(3, 768)


In [ ]:
submission = pd.read_csv('../data/sample_submission.csv', encoding = 'utf-8-sig')

# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('first_submission.csv', index=False, encoding='utf-8-sig')

In [24]:
submission['재발방지대책 및 향후조치계획']

0      ### 답변:\n사고 원인 분석 결과 다음과 같은 재발 방지 대책을 마련할 것을 제...
1      ### 답변:\n1. 절단 및 가공 작업에 대한 사전 안전교육 강화\n   - 작업...
2      ### 답변:\n사고 방지 및 재발 방지 대책으로는 다음과 같은 조치가 필요합니다:...
3      ### 답변:\n주어진 사고 원인에 대응하기 위해 다음과 같은 재발 방지 대책과 향...
4      ### 답변:\n사고 원인 분석 결과, 주된 원인은 점심식사를 위한 이동 시 작업자...
                             ...                        
959    ### 답변:\n안전장비 착용의 철저성 검토와 안전 교육 강화, 석재의 품질 관리 ...
960    ### 답변:\n이 사고를 방지하기 위한 재발 방지 대책과 향후 조치 계획은 다음과...
961    ### 답변:\n1. 사고 원인 분석 및 보고: 사고 발생 원인을 철저히 분석하고 ...
962    주어진 정보를 바탕으로 답변을 드리겠습니다.\n\nA: \n사고 원인 분석 결과, ...
963    ### 답변:\n1. 각도절단기 사용 시 안전모 및 방호덮개 착용의 중요성에 대한 ...
Name: 재발방지대책 및 향후조치계획, Length: 964, dtype: object

In [17]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('submission.csv')

# 대상 ID
target_ids = ['TEST_060', 'TEST_370', 'TEST_701', 'TEST_945']

# 대체할 문장
replacement_text = '작업자 안전교육 실시를 통한 재발 방지 대책 및 향후 조치 계획.'

# 지정된 ID에 대해 해당 열 값 변경
df.loc[df['ID'].isin(target_ids), '재발방지대책 및 향후조치계획'] = replacement_text

# 확인 (선택)
print(df[df['ID'].isin(target_ids)][['ID', '재발방지대책 및 향후조치계획']])

# 필요 시 저장
df.to_csv('submission_filled.csv', index=False, encoding='utf-8-sig')

           ID                       재발방지대책 및 향후조치계획
60   TEST_060  작업자 안전교육 실시를 통한 재발 방지 대책 및 향후 조치 계획.
370  TEST_370  작업자 안전교육 실시를 통한 재발 방지 대책 및 향후 조치 계획.
701  TEST_701  작업자 안전교육 실시를 통한 재발 방지 대책 및 향후 조치 계획.
945  TEST_945  작업자 안전교육 실시를 통한 재발 방지 대책 및 향후 조치 계획.


In [ ]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('submission.csv')

# 추출할 ID 목록
target_ids = ['TEST_060', 'TEST_370', 'TEST_701', 'TEST_945', 'TEST_800']

# 해당 ID의 '재발방지대책 및 향후조치계획' 열 출력
result = df[df['ID'].isin(target_ids)][['ID', '재발방지대책 및 향후조치계획']]

result

,ID,재발방지대책 및 향후조치계획,vec_0
60,TEST_060,NaN,-0.196215
370,TEST_370,NaN,-0.196215
701,TEST_701,NaN,-0.196215
800,TEST_800,"작업 전 안전 점검 강화, 굴착기 작업 숙련도 향상, 작업 공간 안전 확보, 안전 ...",-0.282480
945,TEST_945,NaN,-0.196215


In [10]:
df.isna().sum()

ID                 0
재발방지대책 및 향후조치계획    4
vec_0              0
vec_1              0
vec_2              0
                  ..
vec_763            0
vec_764            0
vec_765            0
vec_766            0
vec_767            0
Length: 770, dtype: int64